In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#https://stackoverflow.com/questions/46910206/extracting-bounding-boxes-and-category-labels-in-ms-coco-dataset
#https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/coco.py

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

import tensorflow as tf
import collections
import random
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

In [ ]:
#littleCo = COCO('/home/r.bohare/coco_data/annotations/instances_train2014.json')

In [ ]:
# Download caption annotation files
annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = True)
  annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
  instances_file = os.path.dirname(annotation_zip)+'/annotations/instances_train2014.json'
  os.remove(annotation_zip)

In [ ]:
#littleCo = COCO('/home/r.bohare/coco_data/annotations/instances_train2014.json')
littleCo = COCO(instances_file)

In [ ]:
id_laptop = littleCo.getCatIds('laptop')
print(id_laptop) #73

id_bag = littleCo.getCatIds('backpack')
print(id_bag) #27

"""Extracting image ids corresponding to backpack and laptop images."""
bag_img_ids = littleCo.getImgIds(catIds=[27])  #3924
laptop_img_ids = littleCo.getImgIds(catIds=[73])  #2475
print("IDs of bag images:", len(bag_img_ids))
print(bag_img_ids)
print("IDs of laptop images:", len(laptop_img_ids))
print(laptop_img_ids)

In [ ]:
bag_laptop_img_ids = littleCo.getImgIds(catIds=[27,73])
print("IDs of bag and laptop images:", len(bag_laptop_img_ids))
print(bag_laptop_img_ids)

In [ ]:
"""Extracting image names corresponding to bag and laptop categories."""
bag_imgs = littleCo.loadImgs(ids=bag_img_ids)
laptop_imgs = littleCo.loadImgs(ids=laptop_img_ids)
print("Bag images:", bag_imgs)
print("Laptop images:", laptop_imgs)

In [ ]:
bag_img_names = [image['file_name'] for image in bag_imgs]
laptop_img_names = [image['file_name'] for image in laptop_imgs]
print("Bag Images:", len(bag_img_names), bag_img_names[:5])
print("Laptop Images:", len(laptop_img_names), laptop_img_names[:5])


In [ ]:
# insert the list to the set
unique_images_set = set(bag_img_names)
print("Unique Bag Images count:", len(unique_images_set))  #3924 so we are getting unique images

In [ ]:
#EXPECTED: #/content/train2014/COCO_train2014_000000246608.jpg
prefix_str = '/content/train2014/'

# Append suffix / prefix to strings in list
final_bag_img_names = [prefix_str + sub for sub in bag_img_names]
print("FINAL Bag Images:", len(final_bag_img_names), final_bag_img_names[:5])

In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
# Mounting the drive
drive.mount('/content/drive')

captions_list = []
for i_name in final_bag_img_names:
  captions_list.append(i_name)

df = pd.DataFrame(captions_list, columns=["Captions"])
df.to_csv('/content/drive/My Drive/Colab Notebooks/_IMAGE CAPTIONING/CAPTIONS_CSV/VAL_PREDICTED_CAPTIONS.csv', index=False)
